In [ ]:
import wandb
import numpy as np
from util import populate_plt_settings, get_column_width, get_fig_size, get_latex_float
from matplotlib import pyplot as plt

In [ ]:
api = wandb.Api()
run = api.run('kennychufk/alluvion/3kqphf8x')
history = run.scan_history()

In [ ]:
num_steps = 2000
vis = np.zeros(num_steps)
bvis = np.zeros(num_steps)
vis_grad = np.zeros(num_steps)
bvis_grad = np.zeros(num_steps)
loss = np.zeros(num_steps)

for row in history:
    step = row['_step']
    if 'vis_real' in row:
        vis[step] = row['vis_real']
    if 'bvis_real' in row:
        bvis[step] = row['bvis_real']
    if '∇vis' in row:
        vis_grad[step] = row['∇vis']
    if '∇bvis' in row:
        bvis_grad[step] = row['∇bvis']
    if 'loss' in row:
        loss[step] = row['loss']

In [ ]:
populate_plt_settings(plt)
plt.rcParams.update({"axes.unicode_minus":False})

In [ ]:
num_psteps = 1200
num_rows = 2
num_cols = 1
fig, axs = plt.subplots(num_rows, num_cols, figsize = get_fig_size(get_column_width(), ratio=2/(np.sqrt(5)-1)))
cmap = plt.get_cmap("tab10")
scale_velocity = 1000
scale_acceleration = 1000
scale_r = 1000
scale_vis = 1e6
scale_loss = 1e6

vis_grad_ax, vis_ax = axs
vis_color = 'tab:blue'
bvis_color = 'tab:red'

vis_grad_ax.plot(vis_grad[:num_psteps], color=vis_color)
vis_grad_ax.set_ylabel(r"$\partial \mathrm{MSE} / \partial \nu_{\mathcal{F}} (s^{-1})$", color = vis_color)
vis_grad_ax.tick_params(axis='y', labelcolor=vis_color)
vis_grad_ax.set_ylim(-3e-6 , 3e-6)
vis_grad_ax.set_xlim(0, num_psteps)

bvis_grad_ax = vis_grad_ax.twinx()
bvis_grad_ax.plot(bvis_grad[:num_psteps], color = bvis_color)
bvis_grad_ax.set_ylabel(r"$\partial \mathrm{MSE} / \partial \nu_{\mathcal{B}} (s^{-1})$", color = bvis_color)
bvis_grad_ax.tick_params(axis='y', labelcolor=bvis_color)
bvis_grad_ax.set_ylim(-0.8e-6, 0.4e-6)
bvis_grad_ax.set_xlim(0, num_psteps)


vis_ax.plot(vis[:num_psteps]*scale_vis, color=vis_color,label=r"$\nu_{\mathcal{F}}$")
vis_ax.set_ylabel(r"SPH boundary viscosity $\nu_{\mathcal{F}}$ (cSt)", color=vis_color)
vis_ax.tick_params(axis='y', labelcolor=vis_color)
vis_ax.set_xlim(0, num_psteps)
vis_ax.set_ylim(2, 3)
vis_ax.set_xlabel('Optimization steps')

bvis_ax = vis_ax.twinx()
bvis_ax.set_ylabel(r"SPH boundary viscosity $\nu_{\mathcal{B}}$ (cSt)", color=bvis_color)
bvis_ax.plot(bvis[:num_psteps]*scale_vis, color=bvis_color)
bvis_ax.tick_params(axis='y', labelcolor=bvis_color)
bvis_ax.set_xlim(0, num_psteps)
bvis_ax.set_ylim(2, 7)

fig.tight_layout(pad=0.05) # should set tight_layout before add_axes()

loss_color = 'tab:green'
loss_ax = fig.add_axes([0.3, 0.15, 0.55, 0.2])
loss_ax.set_ylabel(r"$ \mathrm{MSE}  (mm^2 / s^2)$")
loss_ax.plot(loss[:num_psteps] * scale_loss, color=loss_color)
loss_ax.tick_params(axis='y')
loss_ax.set_ylim(0, 3e-6)
loss_ax.set_xlim(0, num_psteps)


fig.savefig('opt-process.pgf')